# Agent

就其本身而言，语言模型无法执行操作 - 它们只是输出文本。 LangChain的一大用例是创建代理。 代理是使用 LLM 作为推理引擎的系统，用于确定要执行哪些操作以及要传递这些操作的输入。 执行操作后，可以将结果反馈到 LLM 中，以确定是否需要更多操作，或者是否可以完成。

在本教程中，我们将构建一个可以与搜索引擎交互的代理。您将能够向该代理提问，观看它调用搜索工具，并与之进行对话。

## End-to-end agent

下面的代码片段表示一个功能齐全的代理，它使用 LLM 来决定使用哪些工具。它配备了一个通用的搜索工具。它具有对话记忆 - 这意味着它可以用作多轮聊天机器人。

在指南的其余部分，我们将介绍各个组件以及每个部分的作用 - 但如果您只想获取一些代码并开始使用，请随意使用它！

API Reference: [ChatAnthropic](https://api.python.langchain.com/en/latest/chat_models/langchain_anthropic.chat_models.ChatAnthropic.html) | [TavilySearchResults](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html) | [HumanMessage](https://api.python.langchain.com/en/latest/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)

In [ ]:
# Import relevant functionality
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent


In [ ]:

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models import ChatZhipuAI
# zhipuai
os.environ["ZHIPUAI_API_KEY"] = ""
# sparkllm
os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""
#　此处参考：https://www.xfyun.cn/doc/spark/Web.html
os.environ["IFLYTEK_SPARK_API_URL"] = ""
os.environ["IFLYTEK_SPARK_llm_DOMAIN"] = ""
# Tavily AI 
os.environ["TAVILY_API_KEY"] = ""

from langchain_openai import ChatOpenAI
API_SECRET_KEY = ""
BASE_URL = ""  # 代理 base-url 记得加上 /v1

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL

model = ChatOpenAI(temperature=0.9)

In [ ]:

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(temperature=0.9)
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

### LangSmith 
您使用 LangChain 构建的许多应用程序将包含多个步骤，其中包含多次调用 LLM 调用。 随着这些应用程序变得越来越复杂，能够检查您的链或代理内部到底发生了什么变得至关重要。 最好的方法是使用 LangSmith。
在上面的链接上注册后，请确保设置环境变量以开始记录跟踪：

In [15]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

### Tavily

In [16]:
os.environ["TAVILY_API_KEY"] = ""

### Define tools

我们首先需要创建我们想要使用的工具。我们选择的主要工具将是 Tavily - 一个搜索引擎。我们在LangChain中有一个内置工具，可以轻松地使用Tavily搜索引擎作为工具。

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94110/date/2024-7-14', 'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1720941077, 'localtime': '2024-07-14 0:11'}, 'current': {'last_updated_epoch': 1720940400, 'last_updated': '2024-07-14 00:00', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 320, 'wind_dir': 'NW', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c':

### Using Language Models

接下来，让我们学习如何通过调用工具来使用语言模型。LangChain支持许多不同的语言模型，您可以互换使用 - 在下面选择您要使用的语言模型！

In [18]:
from langchain_openai import ChatOpenAI
API_SECRET_KEY = ""
BASE_URL = ""  # 代理 base-url 记得加上 /v1

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL

model = ChatOpenAI(temperature=0.9)

可以通过传入消息列表来调用语言模型。默认情况下，响应是一个字符串。content

In [19]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

我们现在可以看到如何使该模型启用工具调用。为了启用这一点，我们需要让语言模型了解这些工具。bind_tools

In [20]:
model_with_tools = model.bind_tools(tools)

现在我们可以调用模型。首先用一个普通消息调用它，看看它如何响应。我们可以查看字段 content 和 tool_calls。

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


现在，让我们尝试使用一些需要调用工具的输入来调用它。

In [22]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_mdu3sY0BokZcKXkEhuoaoyaK', 'type': 'tool_call'}]


我们可以看到现在没有文本内容，但有一个工具调用！它希望我们调用 Tavily Search 工具。

这还没有调用该工具 - 它只是告诉我们。为了实际调用它，我们需要创建我们的代理。

### Create the agent

现在我们已经定义了工具和 LLM，我们可以创建代理了。我们将使用 LangGraph 来构建代理。 目前，我们正在使用一个高级接口来构建代理，但 LangGraph 的好处是，这个高级接口由一个低级、高度可控的 API 支持，以防你想修改代理逻辑。

现在，我们可以使用 LLM 和工具启动代理。

请注意，我们传入的是 ，而不是 。那是因为会在引擎盖下呼唤我们。`model` `model_with_tools` `create_react_agent` `.bind_tools`


In [23]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### Run the agent

现在，我们可以在一些查询上运行代理！请注意，目前，这些都是无状态查询（它不会记住以前的交互）。请注意，代理将在交互结束时返回最终状态（包括任何输入，我们稍后将看到如何仅获取输出）。

首先，让我们看看当不需要调用工具时它是如何响应的：

In [24]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='51ac3287-3e09-4d07-b82f-c2829597574d'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 85, 'total_tokens': 95}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2efdf059-a789-478c-9149-ad144bb97179-0', usage_metadata={'input_tokens': 85, 'output_tokens': 10, 'total_tokens': 95})]

为了确切地看到引擎盖下发生了什么（并确保它没有调用工具），我们可以看一下 LangSmith 跟踪

现在让我们在一个应该调用该工具的示例中尝试一下

In [25]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='d80dd8e1-3866-4ede-9f34-5558da1bce26'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Zyn0rgbGS6UDbeMADK9NKfip', 'function': {'arguments': '// This function uses the OpenWeather API to fetch the current weather in San Francisco.\nnamespace functions {\n\n  // Returns current weather information for San Francisco\n  export const get_sf_weather = async () => {\n    const response = await fetch(\'https://api.openweathermap.org/data/2.5/weather?q=San Francisco&appid=YOUR_API_KEY_HERE&units=metric\');\n    const json = await response.json();\n    return json;\n  };\n\n} // namespace functions\n\n// USAGE EXAMPLE:\n// const sfWeather = await functions.get_sf_weather();\n// console.log(sfWeather); ("weather" property in the returned JSON contains current weather information)', 'name': 'functions'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 90, 'total_tokens'

### Streaming Messages

我们已经了解了如何调用代理来获得最终响应。如果代理正在执行多个步骤，则可能需要一段时间。为了显示中间进度，我们可以在消息发生时将其流式传输回。.invoke

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

### Streaming tokens

除了流式传输回消息外，流式传输回令牌也很有用。 我们可以用这种方法做到这一点。.astream_events

> 此方法仅适用于 Python 3.11 或更高版本。.astream_events

In [26]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

d:\Program\Anaconda3\envs\langchain-env\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


### Adding in memory

如前所述，此代理是无状态的。这意味着它不记得以前的交互。为了给它内存，我们需要传入一个检查点。传入检查点时，我们还必须在调用代理时传入一个（以便它知道要从哪个线程/对话恢复）。`thread_id`

In [27]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [28]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [29]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 87, 'total_tokens': 98}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b6dcafc0-8c20-45cc-a57b-963ba758833c-0', usage_metadata={'input_tokens': 87, 'output_tokens': 11, 'total_tokens': 98})]}}
----


In [30]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 112, 'total_tokens': 118}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cc52e7f9-78a4-4597-916b-e9cfce799c9a-0', usage_metadata={'input_tokens': 112, 'output_tokens': 6, 'total_tokens': 118})]}}
----


如果我想开始一个新的对话，我所要做的就是改变使用过的thread_id

In [31]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't know your name based on the information available. If you tell me your name, I can address you accordingly.", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4e2ec895-acd7-4504-8feb-fb566c7fe852-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})]}}
----


这就结束了！在此快速入门中，我们介绍了如何创建简单的代理。 然后，我们展示了如何流式传输响应 - 不仅是中间步骤，还有令牌！ 我们还添加了内存，以便您可以与他们进行对话。 代理是一个复杂的话题，有很多东西需要学习！

有关代理的更多信息，请查看 [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph) 文档。这有自己的一套概念、教程和操作指南。